See https://github.com/techleadhd/chatgpt-retrieval/blob/main/chatgpt.py and
[Using ChatGPT with YOUR OWN Data. This is magical. (LangChain OpenAI API)](https://youtu.be/9AXP7tCI9PI?si=Vblh5hzNao4LO_GF) from the [TechLead](https://www.youtube.com/@TechLead)

In [1]:
from langchain.chains import ConversationalRetrievalChain
# In the __init__.py of langchain/libs/langchain/langchain/indexes/
from langchain.indexes import VectorstoreIndexCreator
# This code for DirectoryLoader is in
# langchain/libs/community/langchain_community/document_loaders/directory.py
from langchain_community.document_loaders import DirectoryLoader

In [2]:
from pathlib import Path
import sys

# Make this path be the project's "base" directory, so we can include modules
notebook_directory_ancestor = Path.cwd().resolve().parent.parent
print(notebook_directory_ancestor)
core_code_directory = notebook_directory_ancestor / "CoreCode/"

is_core_code_directory_in_sys_path = str(core_code_directory) in sys.path
is_notebook_directory_ancestor_in_sys_path = str(notebook_directory_ancestor) in sys.path
print("Is CoreCode directory in sys.path?", is_core_code_directory_in_sys_path)
print("Is notebook directory's ancestor in sys.path?", is_notebook_directory_ancestor_in_sys_path)

/InServiceOfX
Is CoreCode directory in sys.path? False
Is notebook directory's ancestor in sys.path? False


In [3]:
if not is_core_code_directory_in_sys_path:
    sys.path.append(str(core_code_directory))

In [4]:
from CoreCode.Utilities.LoadConfigurationFile import LoadConfigurationFile

In [5]:
configuration = LoadConfigurationFile.load_configuration_file()

In [6]:
print(configuration.keys())
finance_path = configuration['BASE_DATA_PATH'] / "Public" / "Finances/"
print(finance_path)
print(finance_path.exists())

dict_keys(['BASE_DATA_PATH'])
/Data/Public/Finances
True


By default, [DirectoryLoader](https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory) uses `UnstructuredLoader` class.

In [7]:
dir_loader = DirectoryLoader(path=finance_path, recursive=True, show_progress=True, use_multithreading=True)
print(dir_loader.path)
print(dir_loader.glob)
print(dir_loader.exclude)
print(dir_loader.loader_cls)

/Data/Public/Finances
**/[!.]*
()
<class 'langchain_community.document_loaders.unstructured.UnstructuredFileLoader'>


In [8]:
vector_store_index = VectorstoreIndexCreator(vectorstore_kwargs={"persist_directory":"chroma_persist"})

/ThirdParty/langchain/libs/core/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [9]:
from CoreCode.Utilities.LoadEnvironmentFile import load_environment_file
load_environment_file()

See `langchain/libs/langchain/langchain/indexes/vectorstore.py`: see that embedding, default factory, is OpenAIEmbeddings. That's why you needed the OpenAI key.

In [10]:
vector_store_index = VectorstoreIndexCreator(vectorstore_kwargs={"persist_directory":"chroma_persist"})

In [11]:
index = vector_store_index.from_loaders([dir_loader,])

  5%|██                                          | 1/22 [00:02<00:53,  2.54s/it]Error loading file /Data/Public/Finances/llc-12-112021.pdf
Error loading file /Data/Public/Finances/llc-1.pdf
Error loading file /Data/Public/Finances/llc-10.pdf
Error loading file /Data/Public/Finances/llc-2-na.pdf
Error loading file /Data/Public/Finances/llc-2.pdf
Error loading file /Data/Public/Finances/Taxes/f1099msc.pdf
Error loading file /Data/Public/Finances/Taxes/i1099gi.pdf
 36%|████████████████                            | 8/22 [00:02<00:03,  3.89it/s]Error loading file /Data/Public/Finances/Taxes/i1040tt.pdf
Error loading file /Data/Public/Finances/Taxes/p5520.pdf
 41%|██████████████████                          | 9/22 [00:02<00:01,  8.30it/s]Error loading file /Data/Public/Finances/Taxes/i1040gi.pdf
Error loading file /Data/Public/Finances/Taxes/f1099div.pdf
 55%|███████████████████████▍                   | 12/22 [00:03<00:01,  8.84it/s]Error loading file /Data/Public/Finances/Taxes/i1099div.pdf

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.